In [ ]:
from jupyter_client import MultiKernelManager
from tqdm.notebook import tqdm

In [ ]:
def get_output(client, msg_id):
    from queue import Empty
    try:
        while True:
            message = client.get_iopub_msg(timeout=10)
            if message["parent_header"]["msg_id"] != msg_id:
                continue
            if message["header"]["msg_type"] != "stream":
                continue
            content = message["content"]
            if content["name"] != "stdout":
                continue
            return content["text"].strip()
    except Empty as e:
        return None

In [ ]:
def get_result(client):
    msg_id = client.execute("print(result)")
    return get_output(client, msg_id)

def get_duration(client):
    msg_id = client.execute("print(duration)")
    return get_output(client, msg_id)

In [ ]:
from pathlib import Path
code = Path("benchmarks/roberta_finetuning.py").read_text()

In [ ]:
mkm = MultiKernelManager()

In [ ]:
num_kernels = 11
kernels = [mkm.get_kernel(mkm.start_kernel()) for _ in range(num_kernels)]
clients = [kernel.client() for kernel in kernels]

In [ ]:
[kernel.provisioner.process.pid for kernel in kernels]

In [ ]:
[kernel.provisioner.process.poll() is None for kernel in kernels]

In [ ]:
results = []
durations = []
for index, client in enumerate(tqdm(clients)):
    client.execute(code, reply=True)
    result = get_result(client)
    duration = get_duration(client)
    results.append(result)
    durations.append(duration)
    tqdm.write(f"{result} {duration}")

In [ ]:
results

In [ ]:
mkm.shutdown_all()

In [ ]:
# len(clients)
# for kernel in kernels:
#     print(kernel.is_alive())
#     for client in clients:
#     print(client.is_alive())